In [7]:
import pandas as pd
import pickle
mdd = []
from tqdm import tqdm
with open('mdd_ncsr_for_ddp.pickle', "rb") as infile: 
    mdd = pickle.load(infile)

In [6]:
caseid = []
time = []
event = []
for x in tqdm(ncsr.index): 
    last_event = 0
    sorted_case = ncsr.iloc[x, 1:].sort_values()
    sorted_vars = list(sorted_case.index)
    count = 0 
    for y in range(len(sorted_case)):
        if sorted_case[y] != -1:
            caseid.append(ncsr.loc[y, 'CASEID'])
            time.append(sorted_case[y])
            event.append(sorted_vars[y])

In [ ]:
add_graphs = []
for case in range(len(comorbid_subset)):
    case_number = add_subset.iloc[case, 0]
    ordered_vals =[]
    ordered_vars = []
    vals = []
    case_age_vars = []
    for symptom in add_subset:
        if symptom != "CASEID":
            if add_subset.loc[case, symptom] > 0:
                vals.append(add_subset.loc[case, symptom])
                case_age_vars.append(symptom)
    # initialize the ordered lists
    #order ints by age symptom occured
    for x in range(0, len(vals)):
        m = vals.index(min(vals))
        ordered_vals.append(vals[m])
        ordered_vars.append(case_age_vars[m])
        del vals[m]
        del case_age_vars[m]

    G = nx.DiGraph()
    for x in range(0, len(ordered_vals)):
        G.add_node(ordered_vars[x], age = ordered_vals[x])
    mult_flag = 0
    back_start = -1
    back_end = -1
    back_flag = 0
    start = 0
    for x in range(0, len(ordered_vars)-1):
        if ordered_vals[x] != ordered_vals[x+1]:
            if not mult_flag:
                G.add_edges_from([(ordered_vars[x], ordered_vars[x+1])])
                back_flag = 0
            else: 
                for y in range(start, x+1):
                    G.add_edges_from([(ordered_vars[y], ordered_vars[x+1])])
                    back_flag = 1
                    back_end = x
            back_start = start
            start = x + 1
            mult_flag = 0
        elif ordered_vals[x] == ordered_vals[x+1]:
            mult_flag = 1
            if back_start<0:
                continue
            if back_flag: 
                for y in range(back_start, back_end + 1):
                    G.add_edges_from([(ordered_vars[y], ordered_vars[x+1])])
            else:
                G.add_edges_from([(ordered_vars[back_start], ordered_vars[x+1])]) 

    add_graphs.append(G)

for x in add_graphs: 
    node_age_added = []
    for from_node in age_variable_subset.iloc[:,0]:
        for to_node in age_variable_subset.iloc[:,0]:
            if x.has_edge(from_node, to_node):
                if from_node not in node_age_added: 
                    onset_count[from_node] += 1
                    average_onset[from_node] += x.nodes[from_node]['age']
                    node_age_added.append(from_node)
                if to_node not in node_age_added:
                    onset_count[to_node] += 1
                    average_onset[to_node] += x.nodes[to_node]['age']
                    node_age_added.append(to_node)
                
                add_edges.loc[from_node, to_node] += 1